<a href="https://colab.research.google.com/github/Shubhra-1/Group9-MI_classification/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setting Seed value
seed_value= 0

# 1. Setting the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Setting the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Setting the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Setting the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

# 5. Configuring a new global `tensorflow` session
from keras import backend as K

session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

# **Neural Network Models**
1. Regular Neural Net
2. CNN
3. Transfer Learning with ResNet50
5. Mobilenet
6. Inceptionnet

In [ ]:
# System imports
import re
import warnings

In [ ]:
# Data science imports
import pandas as pd 
import numpy as np 

In [ ]:
# Visualization imports
import matplotlib.pyplot as plt 

In [ ]:
# sklearn imports
import sklearn.model_selection 
import sklearn.linear_model
import sklearn.ensemble
import sklearn.svm
import sklearn.discriminant_analysis
import sklearn.metrics
from sklearn.pipeline import Pipeline
from sklearn.exceptions import ConvergenceWarning

In [ ]:
# Tensorflow imports
import tensorflow.keras as keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, Dropout, MaxPooling2D, Flatten, Input
import tensorflow.keras.backend as K
from keras.applications import ResNet50

In [ ]:
# Visualization Imports
import visualkeras
from tensorflow.keras import layers, Sequential
from PIL import ImageFont

from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model

In [ ]:
# Helper function to compute metrics

def get_f1(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())

    true_negatives=int(len(y_pred)) - ((int(possible_positives) + int(predicted_positives))-int(true_positives))
    return(f1_val)

def acc(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())

    true_negatives=int(len(y_pred)) - ((int(possible_positives) + int(predicted_positives))-int(true_positives))
    accuracy = (tf.cast(true_positives, tf.float32) + tf.cast(true_negatives, tf.float32)) / tf.cast(len(y_pred), tf.float32)
    return(accuracy)

def prec(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())

    true_negatives=int(len(y_pred)) - ((int(possible_positives) + int(predicted_positives))-int(true_positives))
    return(precision)

def rec(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())

    true_negatives=int(len(y_pred)) - ((int(possible_positives) + int(predicted_positives))-int(true_positives))
    return(recall)

In [ ]:
# Helper class to train sklearn gridsearchcv models & report metrics
class gridsearchcv_model:
    def __init__(self, model, X_train, Y_train, X_val, Y_val, parameter_matrix={}, is_classification=False, cv=4):
        self.is_classification = is_classification
        self.train_model(model, X_train, Y_train, X_val, Y_val, parameter_matrix, cv)
        
    # Trains model using a training set and predicts a validation set
    def train_model(self, model, X_train, Y_train, X_val, Y_val, parameter_matrix={}, cv=4):
        ml_model = sklearn.model_selection.GridSearchCV(model, parameter_matrix, cv=cv, scoring='f1')
     
        ml_model.fit(X_train, Y_train)
        
        self.model = ml_model.best_estimator_
        self.name = re.compile("(.*?)\s*\(").match(str(self.model)).group(1)
        
        self.train = {'name': 'train'}
        self.val = {'name': 'val'}
        
        self.calculate_error(self.train, X_train, Y_train, self.train['name'])
        self.calculate_error(self.val, X_val, Y_val, self.val['name'])
        
        return ml_model
    
    def calculate_error(self, var, X_set, Y_set, name):
        var['name'] = name
        var['predictions'] = self.model.predict(X_set)
        
        var['f1_score'] = sklearn.metrics.f1_score(Y_set, var['predictions'])
        var['accuracy'] = sklearn.metrics.accuracy_score(Y_set, var['predictions'])
        var['precision'] = sklearn.metrics.precision_score(Y_set, var['predictions'])
        var['recall'] = sklearn.metrics.recall_score(Y_set, var['predictions'])
        
        self.print_error(var)
        
    # Prints error metrics
    def print_error(self, var):
        print(self.name + ' ('+ var['name'] + ')')
        
        print("Accuracy: %0.4f" % var['accuracy'])
        print("f1_score: %0.4f" % var['f1_score'])
        print("precision: %0.4f" % var['precision'])
        print("recall: %0.4f" % var['recall'])

## Reading Data

In [ ]:
# Load train and test data into dataframes
df_train_original = pd.read_pickle ("data/epoched_train.pkl")


In [ ]:
# Create column 'pid' which is the patient ID 1 through 9
df_train_original['pid'] = [int(df_train_original['patient_id'][x][2]) for x in range(len(df_train_original))]

# Create column 'trial_id' which is the trial 1 through 3
df_train_original['trial_id'] = [int(df_train_original['patient_id'][x][-2]) for x in range(len(df_train_original))]


In [ ]:
# Dividing the dataset into training, validation and test set 

df_train = df_train_original[df_train_original['trial_id']!=3]
df_train = df_train_original.reindex(np.random.permutation(df_train_original.index)).reset_index(drop=True)

df_val_test = df_train_original[df_train_original['trial_id']==3]
df_val_test = df_val_test.reindex(np.random.permutation(df_val_test.index)).reset_index(drop=True)

df_val = df_val_test[0:int(len(df_val_test)/2)]
df_test = df_val_test[int(len(df_val_test)/2):len(df_val_test)]
df_train_val = df_train.append(df_val)

## Preparation of Data & Adding Gaussian Noise

In [ ]:
# Augment training data by adding Gaussian noise (to avoid overfitting)
for _ in range(1): 
    df_train_augment = df_train.copy()
    for x in ['C3', 'Cz', 'C4', 'EOG:ch01', 'EOG:ch02','EOG:ch03']:
        df_train_augment[x] += np.random.normal(0,1)

    df_train = pd.concat([df_train, df_train_augment])


for _ in range(1): 
    df_train_val_augment = df_train_val.copy()
    for x in ['C3', 'Cz', 'C4', 'EOG:ch01', 'EOG:ch02','EOG:ch03']:
        df_train_val_augment[x] += np.random.normal(0,1)

    df_train_val = pd.concat([df_train_val, df_train_val_augment])

In [ ]:
# Prepare data for training across all subjects
y_train = df_train["event_type"].values.astype(float)
y_val = df_val["event_type"].values.astype(float)
y_train_val = df_train_val["event_type"].values.astype(float)
y_test = df_test["event_type"].values.astype(float)

X_train = df_train.drop(["patient_id", "start_time", "event_type", "pid", "trial_id"], axis=1)
X_val = df_val.drop(["patient_id", "start_time", "event_type", "pid", "trial_id"], axis=1)
X_train_val = df_train_val.drop(["patient_id", "start_time", "event_type", "pid", "trial_id"], axis=1)
X_test = df_test.drop(["patient_id", "start_time", "event_type","pid", "trial_id"], axis=1)

## **1. Neural Network**

In [ ]:
# Concatenate the data for sklearn & neural net models
x_train_nn = np.array(X_train.apply(lambda x:np.concatenate(x), axis=1).values.tolist())
x_val_nn = np.array(X_val.apply(lambda x:np.concatenate(x), axis=1).values.tolist())
x_train_val_nn = np.array(X_train_val.apply(lambda x:np.concatenate(x), axis=1).values.tolist())
x_test_nn = np.array(X_test.apply(lambda x:np.concatenate(x), axis=1).values.tolist())

In [ ]:
# Regular neural net
neural_network = keras.Sequential([Dense(512, activation="relu", kernel_regularizer=keras.regularizers.l2(0.0001)),
                                   Dropout(0.2),
                                   Dense(32, activation="relu", kernel_regularizer=keras.regularizers.l2(0.0001)),
                                   Dropout(0.2),
                                   Dense(1, activation="sigmoid")])

neural_network.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=[get_f1, acc, prec, rec])

history_nn = neural_network.fit(x_train_val_nn, y_train_val, epochs=25, batch_size=64, validation_split=0.2)

In [ ]:
neural_network.summary()

In [ ]:
# Evaluation on test set
neural_network.evaluate(x_test_nn, y_test)

## **2. CNN**

In [ ]:
# Stack the data for CNN models
x_train_cnn = np.array(X_train.apply(lambda x:np.stack(x, axis=-1), axis=1).values.tolist())
x_train_cnn = x_train_cnn.reshape(list(x_train_cnn.shape)+[1])

x_train_val_cnn = np.array(X_train_val.apply(lambda x:np.stack(x, axis=-1), axis=1).values.tolist())
x_train_val_cnn = x_train_val_cnn.reshape(list(x_train_val_cnn.shape)+[1])

x_test_cnn = np.array(X_val.apply(lambda x:np.stack(x, axis=-1), axis=1).values.tolist())
x_test_cnn = x_test_cnn.reshape(list(x_test_cnn.shape)+[1])

In [ ]:
cnn = keras.Sequential([Conv2D(32, (3, 3), activation="relu", input_shape=(1000, 6, 1)),
                        Conv2D(64, (3, 3), activation="relu"),
                        Flatten(),
                        Dense(256, activation="relu"),
                        Dropout(0.2),
                        Dense(128, activation="relu"),
                        Dense(1, activation="sigmoid")])

cnn.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=[get_f1, acc, prec, rec])

history_cnn = cnn.fit(x_train_val_cnn, y_train_val, epochs=15, batch_size=64, validation_split=0.2)

In [ ]:
cnn.summary()

In [ ]:
# Evaluation on test set
cnn.evaluate(x_test_cnn, y_test)

## **3. Resnet**

In [ ]:
# Reshape data for transfer learning models
x_train_transfer = x_train_cnn.reshape((len(x_train_cnn), 50, 40, 3))
x_train_val_transfer = x_train_val_cnn.reshape((len(x_train_val_cnn), 50, 40, 3))
x_test_transfer = x_test_cnn.reshape((len(x_test_cnn), 50, 40, 3))

In [ ]:
resnet = keras.Sequential([ResNet50(include_top=False, weights=None, input_shape=(50,40,3)),
                                     Flatten(),
                                     Dense(1, activation="sigmoid")])

opt = Adam(lr=0.001)

resnet.compile(optimizer=opt, loss='binary_crossentropy', metrics=[get_f1, acc, prec, rec])

history_rn = resnet.fit(x_train_val_transfer, y_train_val, epochs=25, batch_size=64, validation_split=0.2)

In [ ]:
print(resnet.summary())

In [ ]:
# Evaluation on test set
resnet.evaluate(x_test_transfer, y_test)

## **4. MobileNet**

In [ ]:
x_train_mn = x_train_nn.reshape([7360,75,80,1])
x_train_val_mn = x_train_val_nn.reshape([11760,75,80,1])    
x_val_mn = x_val_nn.reshape([720,75,80,1])
x_test_mn = x_test_nn.reshape([720,75,80,1])

In [ ]:
mobile = keras.applications.mobilenet.MobileNet(input_shape=(75, 80, 1),
    weights=None,
    classes=1
    )

mobile.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=[get_f1, acc, prec, rec])
history_mn = mobile.fit(x_train_val_mn, y_train_val, epochs=50, batch_size=64, validation_split=0.2)

In [ ]:
mobile.summary()

In [ ]:
# Evaluation on test set
mobile.evaluate(x_test_mn, y_test)

## **5. Inceptionnet**

In [ ]:
inception = keras.applications.InceptionV3(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(75, 80, 1),
    pooling=None,
    classes=1,
    classifier_activation="softmax"
    )

inception.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=[get_f1, acc, prec, rec])
history_in = inception.fit(x_train_val_mn, y_train_val, epochs=50, batch_size=64, validation_split=0.2)

In [ ]:
inception.summmary()

In [ ]:
# Evaluation on test set
inception.evaluate(x_test_mn, y_test)